환경설정

In [17]:
import os
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import tqdm
import jpype
import ast

In [18]:
folder_path = '/Users/jaesolshin/Documents/GitHub/youtube_dashboard'
file_path = os.path.join(folder_path, 'KPOP_comments_merged_preprocessed_with_nouns.csv')
comments_df = pd.read_csv(file_path)

In [19]:
sample_df = comments_df.copy()
sample_df['word_list'] = sample_df['word_list'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [20]:
sample_df.head()

,Group,Title,comment_type,comment_id_key,parent_id_key,comment,author,date,likes,korean,word_list
0,NMIXX,별별별 (See that),node,UgyiK-QaarUUumBCjUJ4AaABAg,NaN,엔믹스 노래 중독성 있고 좋은데 왜 안뜨는지 진심 의문..,@lumi1216,2024-10-01T13:40:14Z,4,True,"[중독, 진심, 노래, 의문, 엔믹스]"
1,NMIXX,별별별 (See that),node,Ugw_6c4z7iSqIjzfgZB4AaABAg,NaN,대쉬랑 비슷한것 같기도하고 이젠 좀 다른 대중성있는 노래로갔으면하는 아쉬움이있네요 ...,@써니노,2024-10-01T09:56:20Z,1,True,"[기도, 아쉬움, 대쉬, 금도, 대중성, 노래]"
2,NMIXX,별별별 (See that),node,UgxLdDuk-Ji3RKARe_h4AaABAg,NaN,4본부장아 피씨로 엘범 클릭하면 별별별 뮤비로 연결 안되고 수정별인가 그 장면 있는...,@고해진-i4g,2024-10-01T05:16:15Z,4,True,"[간다, 본부장, 엘범, 천만, 한번, 클릭, 연락, 연결, 취해, 피씨, 뮤비, ..."
3,NMIXX,별별별 (See that),node,Ugz-Tnp-d-MLAQSXvQF4AaABAg,NaN,이야 노래 종나 멋있네,@박종한-i9u,2024-09-30T18:33:22Z,10,True,[노래]
4,NMIXX,별별별 (See that),node,UgwlfRfRmVnAsHIREut4AaABAg,NaN,외모췍을 필두로 더더더더 빵빵뜨길...리더로서 해원이 얼마나 노력했을까...멤버들 ...,@egi3581,2024-09-30T17:22:58Z,6,True,"[걸그룹, 멤버, 빵빵, 노력, 묵자, 얼마나, 세대, 외모췍, 로서, 더더, 리더..."


In [21]:
# 새롭게 발견한 불용어 제거
stop_words = [
    "원래", "사실", "세계", "모든", "거기", "관련", "존나", "무슨", "복자", "자신", 
    "빠다", "가지", "기억", "오히려", "다운", "디토", "보기", "이유", "위해", "아예", 
    "타고", "처럼", "여기", "완전", "블랙", "거의", "이전", "차라리", "누구", "바로", 
    "이제", "때문", "그거", "게다가", "일단", "그대로", "시작", "라인", "부터", 
    "본인", "린다", "보임", "소리", "절대", "자체", "얼른", "이후", "진자", "고간", 
    "날자", "마지막", "이야기", "저번", "진진", "그날", "하자", "간다", "조금", 
    "이름", "의미", "매우", "어디", "퍼트", "이기", "건가", "수가", "누가", "수도", 
    "해도", "너희", "얘기", "일이", "한번", "덕분", "그게", "거임", "로서", "라서"
]

# word_list 열에서 stopwords에 포함된 단어들을 제거하는 작업
sample_df['word_list'] = sample_df['word_list'].apply(lambda word_list: [word for word in word_list if word not in stop_words])

In [22]:
correction_dict = {
    "하나": "하나님",
    "아이": "아이브, 아이즈원",
    "믹스": "엔믹스",
    "프티": "피프티",
    "스테이": "스테이씨",
    "보이": "하입보이",
    "버니": "버니즈",
    "뉴진": "뉴진스",
    "수록": "수록곡",
    "비주": "비주얼",
    "그네": "마그네틱",
    "전체": "전체적",
    "대한": "대한민국",
    "침착": "침착맨",
    "아무": "아무리, 아무나",
    "오엠": "오엠지",
    "최고다": "최고",
    "별별": "별별별",
    "너비": "워너비",
    "진영": "박진영",
    "투바": "투바투",
    "홍준": "전홍준",
    "가즈": "가즈아",
    "둥이": "핍둥이, 핌둥이, 엣둥이, 사랑둥이, 켚둥이, 귀염둥이, 뉴둥이, 아입둥이, 엔둥이",
    "현이": "채현, 수현, 지현, 미현, 주현",
    "아이즈": "아이즈원",
    "디오": "아센디오",
    "아센": "아센디오",
    "러브": "러브다이드",
    "국문": "한국문화, 중국문화",
    "대혁명": "문화대혁명",
    "애프터": "애프터라이크",
    "순이": "콩순이",
    "즈하": "카즈하",
    "하이": "하이브",
    "병기": "정병기",
    "스파": "에스파",
    "파가": "에스파",
    "레벨": "레드벨벳, 넥스트레벨",
    "넥스트": "넥스트레벨",
    "파의": "에스파의",
    "테이": "스테이시",
    "에이": "에이셉",
    "입시": "숙대입시",
    "숙대": "숙대입시",
    "배드": "쏘배드",
    "얼빡": "얼빡샷",
    "운지": "다채로운지",
    "티티티": "안티티티",
    "티티": "안티티티",
    "제로": "제로콜라",
    "송이": "음악방송, 팬송",
    "척척": "척척박사, 척척 박사",
    "박사": "척척박사, 척척 박사",
    "코카": "코카콜라",
    "아르": "아르켜줄게",
    "구슬": "유리구슬",
    "스랑": "뉴진스",
    "미드": "미드, 미드소마",
    "소마": "미드소마",
    "레블": "넥스트레블, 넥스트 레블",
    "시대": "소녀시대",
    "아마": "아마겟돈",
    "상미": "영상미",
    "노바": "수퍼노바",
    "파로": "에스파",
    "시혁": "방시혁",
    "마미": "마미손",
    "나야": "에스파는 나야",
    "정호": "정호연",
    "치아": "치아땡",
    "수돌": "통수돌",
    "사랑": "사랑해"
}


In [23]:
# word_list 열에서 correction_dict의 키를 찾아서 대체 및 제거하는 함수 정의
def replace_or_remove_keywords(row):
    word_list = row['word_list']
    comment = row['comment']

    # 새로운 word_list를 저장할 리스트
    new_word_list = []

    # word_list에서 correction_dict의 키를 확인하고 대체 또는 제거
    for word in word_list:
        if word in correction_dict:  # word가 correction_dict의 키인지 확인
            possible_replacements = correction_dict[word]
            
            # replacement가 여러 개일 경우 분할
            if ',' in possible_replacements:
                possible_replacements = possible_replacements.split(', ')
            else:
                possible_replacements = [possible_replacements]
            
            # comment에서 각 replacement가 있는지 확인
            replaced = False
            for replacement in possible_replacements:
                if replacement in comment:
                    new_word_list.append(replacement)  # 대체할 값이 있으면 추가
                    replaced = True
                    break  # 대체가 이루어지면 반복 중단
            
            # 대체가 이루어지지 않으면 word를 제거 (추가하지 않음)
        else:
            # word가 correction_dict에 없으면 유지
            new_word_list.append(word)

    return new_word_list


# word_list 열을 수정하는 apply 함수
sample_df['word_list'] = sample_df.apply(replace_or_remove_keywords, axis=1)

In [24]:
# 특정 단어가 포함된 댓글 조회하기

def search_from_comment(hot_word, dataframe=sample_df, likes=False, n_comments=10, random_state=42):

    # 데이터 프레임에서 hot_word를 포함하고 있는 행 필터링
    filtered_df = dataframe[dataframe['comment'].str.contains(hot_word, na=False)]
    print(len(filtered_df))

    # 좋아요 기준 정렬
    if likes:
        filtered_df = filtered_df[['comment','word_list','likes']].sort_values(by='likes', ascending=False)
        result = filtered_df[:n_comments]
    
    # 랜덤으로 조회
    else :
        result = filtered_df[['comment','word_list','likes']].sample(n = n_comments, random_state=42)
    
    return result


# 특정 단어가 포함된 댓글 조회하기
def search_from_word_list(hot_word, dataframe=sample_df, likes=False, n_comments=10, random_state=42):

    # 데이터 프레임에서 hot_word를 포함하고 있는 행 필터링
    filtered_df = dataframe[dataframe['word_list'].apply(lambda x: hot_word in x)]
    print(f"필터링된 댓글 수: {len(filtered_df)}")

    # 필터링된 결과가 없을 경우 빈 데이터프레임 반환
    if len(filtered_df) == 0:
        return pd.DataFrame(columns=['comment', 'word_list', 'likes'])
    
    # 좋아요 기준 정렬
    if likes:
        filtered_df = filtered_df[['comment', 'word_list', 'likes']].sort_values(by='likes', ascending=False)
        result = filtered_df[:n_comments]
    
    # 랜덤으로 조회
    else:
        result = filtered_df[['comment', 'word_list', 'likes']].sample(n=n_comments, random_state=random_state)
    
    return result

In [25]:
search_from_word_list('아이브', likes=True)

필터링된 댓글 수: 27121


,comment,word_list,likes
587608,아이브의 케이팝 분위기는 정말 대단해요. 매번 우리를 놀라게 해요.,"[케이팝, 아이브, 매번, 분위기]",7492
610875,한창 해외판으로 나가는 돌판인데 아이브는 역시 아이브구나.. 한국풍으로 기강 딱 잡겠네,"[한창, 돌판, 한국, 해외, 기강, 아이브, 구나]",4385
581639,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 진짜 초등학생들 취향저격일듯.. 나 초딩이엇으면 아이브놀이 햇...,"[참고, 재밋, 저격, 봣슴니, 초딩, 엇으, 초등학생, 취향, 성인, 아이브]",3672
187932,아이브 꽃길만 걷자 !! 절대 지켜!!,"[아이브, 꽃길]",3316
613545,우와.. 아니 어떻게 아이브 멤버들은 다 매달 컴백마다 리즈를 찍는거지,"[매달, 리즈, 멤버, 아이브, 컴백]",3094
580157,내가보기엔 맨처음에 물속에서 가을이를 부르던 아이브가 흑이브임. 유독 스산하게 연출...,"[아센디오, 유독, 불가능, 아센디오, 조종, 스산하, 흑화, 아이브, 는걸, 시전...",2712
607776,자기복제 없이 매번 새로운 색깔로 도전하는 거 진짜 호감 ㅋㅋ 그렇다고 아이브만의 ...,"[복제, 호감, 매번, 정체, 도전, 아이브, 색깔]",2646
614388,스타쉽이 드디어 아이브 멤버 각각의 보컬을 살리기 시작한 듯 진짜 딱 본인들한테 맞...,"[스타, 파트, 리즈, 튀어, 듣기, 음색, 각각, 멤버, 보컬, 아이브]",2645
614267,와 티져듣고 이번에는 평범하게 후크송으로 가려나보다 했는데 진짜 강렬하네 아이브는 ...,"[복제, 후크송, 앨범, 티져, 아이브]",2154
578266,초딩 이야기 많이 하지만 정작 이런 컨셉은 마법소녀에 많이 빠져살던 30대 정도가 ...,"[걸그룹, 초딩, 가족, 마법소녀, 컨셉, 아이브, 정작]",1565


In [26]:
sample_df.shape

(648060, 11)

In [27]:
sample_df.to_csv('KPOP_comments_merged_preprocessed_after_kw_cleaning.csv', encoding='utf-8-sig', index=False)

# 애칭은 나중에..

In [28]:
import re

# 애칭과 본명을 매칭시키는 딕셔너리 정의 (그룹별로 애칭 포함)
nickname_to_name_dict = {
    '뉴진스': {
        '팜하니': '하니', '빵하니': '하니', '뿅오리': '하니', '빠마니': '하니', '아기토끼': '하니',
        '팜토끼': '하니', '파마니': '하니', '팜투리': '하니', '응옥이': '하니', '옥흔이': '하니',
        '팜팜언니': '하니', '팜팜팜': '하니', '감자': '하니', '팜후드': '하니', '팜자후': '하니',
        '코코팜': '하니', '포도알': '하니', '동그라미': '하니', '폼하니': '하니', '팜 미쳤다': '하니',
        '판다팜': '하니', '물개팜': '하니', '하니비': '하니', '팜마리오': '하니', '팜자님': '하니',
        '수달팜': '하니', '팜호초': '하니', '팜다이크': '하니',
        # 다른 뉴진스 멤버 애칭 추가 가능
    },
    # 다른 그룹의 애칭과 본명도 추가 가능
}

# 그룹별 멤버 리스트 정의
group_members = {
    'NewJeans': ['하니', '민지', '다니엘', '혜인', '해린'],
    # 다른 그룹의 멤버도 추가 가능
}

# 댓글에서 애칭을 확인하고 member 열에 본명 추가하는 함수 정의
def add_real_name_to_member_list(row):
    comment = row['comment']
    member_list = row['member'] if isinstance(row['member'], list) else []  # member 열이 없을 경우 빈 리스트로 초기화
    group = row['Group']  # 각 댓글이 속한 그룹

    # 해당 그룹의 멤버만 처리
    if group in nickname_to_name_dict:
        for nickname, real_name in nickname_to_name_dict[group].items():
            # 정규표현식: 단어 경계 (\b) 또는 뒤에 붙는 한국어 조사 인식 (야, 는, 를, 가, 등)
            if re.search(rf'{re.escape(nickname)}([^\w]|$)', comment) and real_name not in member_list:
                # 해당 그룹의 멤버인 경우에만 추가
                if real_name in group_members[group]:
                    member_list.append(real_name)

    return member_list

# sample_df에 member 열이 없으면 새로 생성
if 'member' not in sample_df.columns:
    sample_df['member'] = [[] for _ in range(len(sample_df))]

# sample_df에 대해 apply를 사용하여 member 열 업데이트
sample_df['member'] = sample_df.apply(add_real_name_to_member_list, axis=1)


In [29]:
search_from_comment('사랑', likes=True)

28606


,comment,word_list,likes
358210,어른들의 비겁함에 너희의 청춘이 아프지 않기를 바라 ..... 뉴진스 사랑해 계속 ...,"[뉴진스, 어른, 청춘, 사랑해]",27987
359863,뉴진스는 뉴진스만의 청량 청순 학생 분위기가 있어서 보고만 있어도 막 설레더라… 진...,"[뉴진스, 청량, 청순, 학생, 분위기]",12341
313078,세련되고 힙하게 신나는데 시끄럽진 않음 뉴진스가 말아주는 따뜻한 아아메 기가막힌다 ...,"[뉴진스, 힙하, 사랑해, 화이팅, 아아]",6694
332959,하니가 아기소 조아해서 장면 넣어달라고 했다던데.. 정말 대표님 멤버들 지독하게 아...,"[대표, 하니, 멤버, 달라, 송아지, 아기, 장면]",6567
295488,뉴진스가 또 뉴진스했다 너희를 사랑하는 사람이 이렇게 많다는 걸 알아줬으면 좋겠어 ...,"[뉴진스했다, 뉴진스, 가득, 응원글, 삭제, 하이브]",6532
492720,1.민희진이 하이브에 들어감 2.근데 알고보니 하이브가 단월드사이비 3.하지만 무르...,"[신의, 단월드, 무엇, 먹음, 타이틀, 뉴진스들, 단군신화, 잡음, 방시혁, 통수...",6510
423004,어떤 회사처럼 어쭙잖게 카피하지도 못하게 항상 독보적이고 실험적인 컨셉에 도전하는 ...,"[독보, 개처못, 도전, 카피, 순간, 실험, 지존, 슴빠하, 컨셉, 회사]",6254
334395,와... 타이틀 아닌데도 대박이다 오늘만 몇번째 듣는건지.. 계속 맴돌아서 틀어놓고...,"[화보, 타이틀, 프로필, 거리, 응원, 트렌드, 노래, 아이돌, 컨셉, 독보, 출...",4856
643735,아니 전체적으로 뮤비컬도 엄청 높아진 것 같고 파트 분배도 엄청 잘 된 것 같고 노...,"[노래, 분배, 파트, 메이크업, 뮤비, 스테이씨, 색감, 전체적, 가사, 역대]",4507
502838,방시혁 단월드 백퍼라고 생각하신다면 본김에 댓 안사라지게 좋아요 눌러주신다면 감사드...,"[시장, 존재, 문화, 뉴진스, 하이브, 위인, 시절, 단월드, 아이돌, 인기, 감...",3816


In [30]:
sample_df[sample_df['Group']=='NewJeans']

,Group,Title,comment_type,comment_id_key,parent_id_key,comment,author,date,likes,korean,word_list,member
24917,NewJeans,'Hype Boy' Official MV (HANNI ver.),node,UgxQhPmB7WZNIjKELJR4AaABAg,NaN,노페이스 언니가 흰색옷 붉은색으로 염색한 눈썹 즉 단월드를 상징하는 겉모습을 하고있...,@won905,2024-09-22T16:24:42Z,1,True,"[상징, 뉴진스, 눈썹, 단월드, 겉모습, 노페, 존재, 표현, 흰색, 이스, 당시...",[]
24918,NewJeans,'Hype Boy' Official MV (HANNI ver.),reply,NaN,UgxQhPmB7WZNIjKELJR4AaABAg,그리구 하니가 플러스글로벌오디션에 직접 지원한게 노페이스에게 직접 팬이라고 연락한 ...,@won905,2024-09-22T16:29:58Z,1,True,"[상징, 연락, 하니, 뉴진스, 멤버, 노페, 직접, 플러스, 이스, 생존, 원한,...",[]
24919,NewJeans,'Hype Boy' Official MV (HANNI ver.),node,UgyDzAV-30Cy8lXk8J94AaABAg,NaN,인사 씹으면 크닐남...,@jamespark1582,2024-09-21T17:25:20Z,0,True,"[크닐남, 인사]",[]
24920,NewJeans,'Hype Boy' Official MV (HANNI ver.),node,UgzE_gjT223fn9b3JuN4AaABAg,NaN,하니 귀여웡,@jelee2798,2024-09-20T13:10:41Z,0,True,"[하니, 귀여웡]",[]
24921,NewJeans,'Hype Boy' Official MV (HANNI ver.),node,Ugy9TXutM2HSKNwu0vh4AaABAg,NaN,하니 미소가 정말 사람 기분 좋게 한다,@nero_sena,2024-09-17T15:36:11Z,0,True,"[기분, 하니, 미소]",[]
...,...,...,...,...,...,...,...,...,...,...,...,...
547432,NewJeans,Ditto (side A),node,UgyRySjsGtVK8PwPvBB4AaABAg,NaN,1등,@응사마-h6c,2022-12-19T09:00:18Z,1,True,[],[]
547433,NewJeans,Ditto (side A),node,UgwlpFTp85LoCbwM6Q54AaABAg,NaN,와,@paaaQ,2022-12-19T09:00:16Z,0,True,[],[]
547434,NewJeans,Ditto (side A),node,UgzOcznxvfrNHRGFGmd4AaABAg,NaN,와,@0x197,2022-12-19T09:00:16Z,1,True,[],[]
547435,NewJeans,Ditto (side A),node,Ugwewh_EFJDcFEubxDR4AaABAg,NaN,와 뉴깅이들 신곡이다 ㅠㅠㅠ,@ZB1-peach,2022-12-19T09:00:11Z,2,True,"[신곡, 뉴깅이들]",[]
